### Target:

- Setup the notebook with proper local imports
- Download CIFAR10
- Setup dataloader for dataset
- Print Statistics
- Display samples
- Display augmentations
- Train Model
- Display misclassified data

### Results:

- Parameters: 
- Best Training Accuracy: 
- Best Test Accuracy: 

### Analysis:



## Import Modules

In [6]:
# Import all the required modules
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

if 'google.colab' in sys.modules:
    # Download the repo from GitHub
    print("Downloading repository on Colab...")
    !git clone https://github.com/Shilpaj1994/ERA.git
    
    # Import files from the downloaded repository
    sys.path.insert(0,'./ERA/Session8/')

ModuleNotFoundError: No module named 'torch'

## Transformations

In [2]:
# Train Phase transformations
train_set_transforms = {
    'standardize': transforms.ToTensor(),
#     'centercrop': transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
#     'affine': transforms.RandomAffine(degrees=(-7, 7), translate=(0.1, 0.1), scale=(0.09, 1.0)),
    'randomcrop': transforms.RandomResizedCrop((28, 28), (0.85, 1.0)),
    'rotation': transforms.RandomRotation((-7., 7.), fill=0),
    'normalize': transforms.Normalize((0.1307,), (0.3081,)),
}

# Test Phase transformations
test_set_transforms = {
    'standardize': transforms.ToTensor(),
    'resize': transforms.Resize((28, 28)),
    'normalize': transforms.Normalize((0.1325,), (0.3104,))
}

train_transforms = transforms.Compose(train_set_transforms.values())
test_transforms = transforms.Compose(test_set_transforms.values())

NameError: name 'transforms' is not defined

## Dataset and Creating Train/Test Split

In [3]:
train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

NameError: name 'datasets' is not defined

## Dataloader Arguments & Test/Train Dataloaders

In [ ]:
SEED = 8

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

## Data Statistics

In [ ]:
from utils import get_data_statistics

get_data_statistics(train, train_loader)

## Visualize Training Data

In [ ]:
from utils import get plot_data

plot_data(images, number_of_samples=60)

## Visualize Augmentations

In [ ]:
from utils import visualize_augmentation

visualize_augmentation(train_loader, train_set_transforms)

## Model and its Parameters

In [4]:
from model import Net
from utils import get_summary

get_summary(Net(), (1, 28, 28))

ModuleNotFoundError: No module named 'torch'

## Model Training and Testing

In [ ]:
from utils import train, test

# Data to plot accuracy and loss graphs
train_losses = []
test_losses = []
train_acc = []
test_acc = []

test_incorrect_pred = {'images': [], 'ground_truths': [], 'predicted_vals': []}

# Put the model on selected device
model = Net().to(device)

# Optimization algorithm to update the weights
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Scheduler to change the learning rate after specific number of epochs
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)

# New Line
criterion = F.nll_loss

# Number of epochs for which model is to be trained
num_epochs = 20

# For each epoch
for epoch in range(1, num_epochs+1):
    print(f'Epoch {epoch}')

    # Train the model on training dataset and append the training loss and accuracy
    correct, processed, train_loss = train(model, device, train_loader, optimizer, criterion)
    train_acc.append(100 * correct / processed)
    train_losses.append(train_loss / len(train_loader))

    # Test the model's performance on test dataset and append the training loss and accuracy
    correct, test_loss = test(model, device, test_loader, criterion)
    test_acc.append(100. * correct / len(test_loader.dataset))
    test_losses.append(test_loss)
    
    # Update the learning rate after specified number of epochs
#     scheduler.step()

## Visualize Train and Test - Loss and Accuracies

In [ ]:
# Print loss and accuracy
from utils import display_loss_and_accuracies

display_loss_and_accuracies(train_losses, train_acc, test_losses, test_acc)

## Visualize Misclassified Images

In [ ]:
from utils import get_misclassified_data

misclassified_data = get_misclassified_data(model, device, test_loader)

plot_data(misclassified_data, 30)